In [1]:
println("Hello World!")

Hello World!


In [2]:
new java.io.File("/data/shakespeare").list

[merrywivesofwindsor, twelfthnight, midsummersnightsdream, loveslabourslost, asyoulikeit, comedyoferrors, muchadoaboutnothing, tamingoftheshrew]

# Scala для Spark (и этого достаточно)


[Scala](http://scala-lang.org) обязательный инструмент для больших данных, особенно для [Apache Spark's](http://spark.apache.org) Scala API в Spark создает более явные и простые конструкции.

## Почему Scala?

4 пункта за Scala:
* **Performance:** Spark написан на Scala, основное API написано для Scala, то есть самое полное и содержательное API, особенно для [RDD](http://spark.apache.org/docs/latest/programming-guide.html#resilient-distributed-datasets-rdds). Spark на Scala исключает шаги по переводу Python кода для JVM.
* **Debugging:** Ошибки Runtime и общее понимание ошибок проще, когда вы знакомы со Scala, так как информация из debugger приходит в виде Scala Exception.
* **Concise, Expressive Code:** Решения на Scala более короткие и локаничные (особено, по сравнению с Java). На Scala вы будете более продуктивны.
* **Type Safety:** Есть статическая типизация, что не позволяет не совершать ошибок и не получать не ожиданных результатов.

### У Scala есть минусы?

В Scala есть минусы (особенно, если очень плохо с функциональным программированием и понимание [жаргона](https://github.com/hemanth/functional-programming-jargon)):
* **Libraries:** Scala не так богата библиотеками, как Python.
* **Advanced Language Features:** В Scala много особенностей, которые можно не знать и из-за этого не использовать.


**! Note:** всегда можно научится функциональному программированию. [Всё о ФП](https://github.com/xgrommx/awesome-functional-programming#tutorials-and-articles)

### Материалы, которые вам помогут

* [Programming Scala, Second Edition](http://shop.oreilly.com/product/0636920033073.do)
* [Scala Language Website](http://scala-lang.org/)

* Scaladocs для <a href="http://www.scala-lang.org/api/current/#package" target="scala_scaladocs">Scala</a>.
* Scaladocs для <a href="http://spark.apache.org/docs/latest/api/scala/index.html#package" target="spark_scaladocs">Spark</a>.

### Рабочая среда

In [3]:
// конфигурирование среды для настройки типов (всегда отображать)
%showTypes on

Types will be printed.


[SparkContext](http://spark.apache.org/docs/latest/programming-guide.html#initializing-spark) - это точка входа (первый шаг работы) в среде Spark, даже если вы работаете с `SparkSession`. 

Вы указываете: где ваш кластре, какая его конфигурация и с какими настройками мы будем рабоать.

Часто перемунню, где вызывают `SparkContext` называют `sc`

In [4]:
sc

Spark local-1529932883293: Some(http://638dd926b4fa:4040)

In [5]:
println("Spark version:      " + sc.version)
println("Spark master:       " + sc.master)
println("Running 'locally'?: " + sc.isLocal)

Spark version:      2.3.0
Spark master:       local[*]
Running 'locally'?: true


## Изучение Scala через работу со Spark

> **Note:** "method" vs. "function"  (чем отличается?)

Сделаем погружение в синтаксис Scala на примере функции, которая выводить информационное сообщение

In [6]:
/*
 * "info" принимает String аргумент и печатает строку, которую она получила
 */
def info(message: String): String = {
    println(message)

    // последняя строка функции - это то, что будет она взвращать
    // "return" не обязательное слово
    message  
}

info: (message: String)String


In [7]:
/*
 * "error" принимает String аргумент и печатает строку
 */
def error(message: String): String = {   
    
    val fullMessage = s"""
        |********************************************************************
        |
        |  ERROR: $message
        |
        |********************************************************************
        |""".stripMargin
    println(fullMessage)
    
    fullMessage
}

error: (message: String)String


In [8]:
val infoString = info("All is well.")

All is well.


infoString = All is well.


All is well.

In [9]:
val errorString = error("Uh oh...")


********************************************************************

  ERROR: Uh oh...

********************************************************************



errorString = 


"
"



********************************************************************

  ERROR: Uh oh...

********************************************************************


In [10]:
errorString

"
"



********************************************************************

  ERROR: Uh oh...

********************************************************************


в Scala есть особенное и интересное форматирование строк:

* **Тройные кавычки:** `"""..."""` - для многострочного сообщения
* **Строка с возмжожностью форматирования (с переменными):** `s` в самом начале, перед кавычками, `s"..."` или `s"""..."""`, позволит вам использовать переменные из среды прямо в предложениях

In [11]:
s"""Use braces for expressions: ${sc.version}.
You can omit the braces when just using a variable: $sc
However, watch for ambiguities like ${sc}andextrastuff"""

Use braces for expressions: 2.3.0.
You can omit the braces when just using a variable: org.apache.spark.SparkContext@3ca367f9
However, watch for ambiguities like org.apache.spark.SparkContext@3ca367f9andextrastuff

`stripMargin` - методл для управлениями проблемами в строке. Если в строку `|`, то вы можете указывать на количество пробелов перед текстом, относительно левого края.

In [12]:
s"""
    |line 1
    |  line 2
    |  |  line 3
    |""".stripMargin

"
"



line 1
  line 2
  line 3


### Mutable Variables (Var) vs. Immutable Values (Val)

* `val immutableValue = ...` - переменная, которую нельзя перезаписать, то есть это `immutableValue` (рекомендуемый вариант)
* `var mutableVariable = ...` - обычный тип переменных, которые можно перезаписывать, так как вы привыкли в Python

### Добавим файл

Так как Scala компилируется в JVM, то мы можем использовать общие переменные и библиотеки.

Для работы с файлами (для чтения и записи) мы будем использовать библиотеку [java.lang.String](https://docs.oracle.com/javase/8/docs/api/java/lang/String.html) и [java.io.File](https://docs.oracle.com/javase/8/docs/api/java/io/File.html)

In [15]:
import java.io.File

In [ ]:
val shakespeare = new File("/data/shakespeare")

В Scala `if` конструкция - это выражние на результат True или False (в Java / Python - это условные конструкции).

То есть, если `if` условие возвращает `true` или `false`, то результат возвращается в `success` переменную, которую мы можем сразу использовать далее

In [ ]:
val success = if (shakespeare.exists == false) {   // doesn't exist already?
    error(s"Data directory path doesn't exist! $shakespeare")  // ignore returned string
    false
} else {
    info(s"$shakespeare exists")
    true
}
println("success = " + success)

In [ ]:
val pathSeparator = File.separator
val targetDirName = shakespeare.toString
val plays = Seq(
    "tamingoftheshrew", "comedyoferrors", "loveslabourslost", "midsummersnightsdream",
    "merrywivesofwindsor", "muchadoaboutnothing", "asyoulikeit", "twelfthnight")

if (success) {
    println(s"Checking that the plays are in $shakespeare:")
    val failures = for {
        // for play in plays
        play <- plays
        
        // создаем строку
        playFileName = targetDirName + pathSeparator + play
        
        // чтение файла
        playFile = new File(playFileName)
        if (playFile.exists == false)
    } yield {
        s"$playFileName:\tNOT FOUND!"
    }
  
    println("Finished!")
    if (failures.size == 0) {
        info("All plays found!")
    } else {
        println("The following expected plays were not found:")
        failures.foreach(play => error(play))
    }
}

## Функции, как аргументы

Сделаем разбор на примере `collection.foreach(println)`

In [19]:
println("Pass println as the function to use for each element:")
plays.foreach(println)

println("\nUsing an anonymous function that calls println: `str => println(str)`")
println("(Note that the type of the argument `str` is inferred to be String.)")
plays.foreach(str => println(str))

println("\nAdding the argument type explicitly. Note that the parentheses are required.")
plays.foreach((str: String) => println(str))

println("\nWhy do we need to name this argument? Scala lets us use _ as a placeholder.")
plays.foreach(println(_))

println("\nFor longer functions, you can use {...} instead of (...).")
println("Why? Because it gives you the familiar multiline block syntax with {...}")
plays.foreach {
  (str: String) => println(str)
}

println("\nThe _ placeholder can be used *once* for each argument in the list.")
println("As an assume, use `reduceLeft` to sum some integers.")
val integers = 0 to 10   // python range
integers.reduceLeft((i,j) => i+j)
integers.reduceLeft(_+_)

Pass println as the function to use for each element:
tamingoftheshrew
comedyoferrors
loveslabourslost
midsummersnightsdream
merrywivesofwindsor
muchadoaboutnothing
asyoulikeit
twelfthnight

Using an anonymous function that calls println: `str => println(str)`
(Note that the type of the argument `str` is inferred to be String.)
tamingoftheshrew
comedyoferrors
loveslabourslost
midsummersnightsdream
merrywivesofwindsor
muchadoaboutnothing
asyoulikeit
twelfthnight

Adding the argument type explicitly. Note that the parentheses are required.
tamingoftheshrew
comedyoferrors
loveslabourslost
midsummersnightsdream
merrywivesofwindsor
muchadoaboutnothing
asyoulikeit
twelfthnight

Why do we need to name this argument? Scala lets us use _ as a placeholder.
tamingoftheshrew
comedyoferrors
loveslabourslost
midsummersnightsdream
merrywivesofwindsor
muchadoaboutnothing
asyoulikeit
twelfthnight

For longer functions, you can use {...} instead of (...).
Why? Because it gives you the familiar multiline

integers = Range(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


55

# Spark 

## Inverted Index


Inverted Index - это очень сложный процесс, когда вы "проходитесь" по всему корпусу документов, токенизируете слова и возвращаете список слов и индекс. Это подобие word count, но более сложное.

In [20]:
val iiFirstPass1 = sc.wholeTextFiles(shakespeare.toString).
    flatMap { location_contents_tuple2 =>     // применяем функцию, которая создает pair rdd (слово, документ)
        val words = location_contents_tuple2._2.split("""\W+""")
        val fileName = location_contents_tuple2._1.split(pathSeparator).last
        words.map(word => ((word, fileName), 1))
    }.
    reduceByKey((count1, count2) => count1 + count2).
    map { word_file_count_tup3 => 
        (word_file_count_tup3._1._1, (word_file_count_tup3._1._2, word_file_count_tup3._2)) 
    }.
    groupByKey.
    sortByKey(ascending = true).
    mapValues { iterable => 
        val vect = iterable.toVector.sortBy { file_count_tup2 => 
                  (-file_count_tup2._2, file_count_tup2._1)
        }
        vect.mkString(",")
    }

iiFirstPass1 = MapPartitionsRDD[9] at mapValues at <console>:44


MapPartitionsRDD[9] at mapValues at <console>:44

Сделаем пошаговый разбор

In [ ]:
val fileContents = sc.wholeTextFiles(shakespeare.toString)
fileContents   //

In [22]:
// сделаем разбор типов (особенно вложенных ("bar", 202L))
("foo", 101, 3.14159, ("bar", 202L))

(foo,101,3.14159,(bar,202))

Токенизация

> **Note:**  "tokenization", в данном примере очень простая, мы не делаем "глубокое очищение слов". Так как токенизация - это основной элемент процесса _natural language processing_ (NLP), то нужно будет отдельно узнать, как правильно сделать слова "чистыми"

In [25]:
val wordFileNameOnes = fileContents.flatMap { location_contents_tuple2 => 
    // record: (file_path, "all the words in the file")
    // mytuple._2 => дай мне только второй элемент
    val words = location_contents_tuple2._2.split("""\W+""")              
    // mytuple._1 => дай мне только первый элемент
    val fileName = location_contents_tuple2._1.split(pathSeparator).last  
    // создаем новый кортеж  и подставляем базовое числов для счета
    words.map(word => ((word, fileName), 1))
}
wordFileNameOnes

wordFileNameOnes = MapPartitionsRDD[12] at flatMap at <console>:34


MapPartitionsRDD[12] at flatMap at <console>:34

Примеры использования

`argument_list => body`:

```scala
location_contents_tuple2 => 
    val words = ...
    ...
}
```


```scala
(some_tuple3: (String,Int,Double)) => ...
(arg1, arg2, arg3) => ...
(arg1: String, arg2: Int, arg3: Double) => ...
```

Подробнее про `location_contents_tuple2`

Мы берем `первый` элемент `_1` и `второй` способом `_2` (в Python [n])


```scala
val words = location_contents_tuple2._2.split("""\W+""")
```

Если известно, что элемент последний, то можно использовать метод `.last`

```scala
val fileName = location_contents_tuple2._1.split(pathSeparator).last
```

In [26]:
wordFileNameOnes.count

173336

In [27]:
wordFileNameOnes.take(10).foreach(println)

((,merrywivesofwindsor),1)
((THE,merrywivesofwindsor),1)
((MERRY,merrywivesofwindsor),1)
((WIVES,merrywivesofwindsor),1)
((OF,merrywivesofwindsor),1)
((WINDSOR,merrywivesofwindsor),1)
((DRAMATIS,merrywivesofwindsor),1)
((PERSONAE,merrywivesofwindsor),1)
((SIR,merrywivesofwindsor),1)
((JOHN,merrywivesofwindsor),1)


Теперь сделаем отбор уникальных пар `(word,fileName)`

In [28]:
val uniques = wordFileNameOnes.reduceByKey((count1, count2) => count1 + count2)
uniques

uniques = ShuffledRDD[13] at reduceByKey at <console>:36


ShuffledRDD[13] at reduceByKey at <console>:36

In [29]:
uniques.count

27276

In [30]:
uniques.take(30).foreach(println)

((dexterity,merrywivesofwindsor),1)
((force,muchadoaboutnothing),2)
((whole,comedyoferrors),2)
((lamb,muchadoaboutnothing),2)
((blunt,tamingoftheshrew),3)
((letter,merrywivesofwindsor),19)
((crest,asyoulikeit),1)
((bestow,asyoulikeit),1)
((rear,midsummersnightsdream),1)
((crossing,tamingoftheshrew),1)
((wronged,merrywivesofwindsor),4)
((S,tamingoftheshrew),10)
((HIPPOLYTA,midsummersnightsdream),19)
((revolve,twelfthnight),1)
((er,merrywivesofwindsor),11)
((renown,asyoulikeit),1)
((cubiculo,twelfthnight),1)
((All,twelfthnight),3)
((power,loveslabourslost),8)
((Albeit,asyoulikeit),1)
((lips,tamingoftheshrew),3)
((upshot,twelfthnight),1)
((approach,midsummersnightsdream),4)
((mean,muchadoaboutnothing),5)
((embossed,asyoulikeit),1)
((varnish,loveslabourslost),2)
((Apollo,midsummersnightsdream),1)
((spangled,midsummersnightsdream),1)
((gentlemen,comedyoferrors),1)
((Rebuke,loveslabourslost),1)


_inverted index_ подразуменвает группировку по слову и файл `((word,fileName),count)`, но счет должен быть по конкретному документу `(word,(fileName,count))`

In [31]:
val words = uniques.map { word_file_count_tup3 => 
    (word_file_count_tup3._1._1, (word_file_count_tup3._1._2, word_file_count_tup3._2)) 
}

words = MapPartitionsRDD[14] at map at <console>:38


MapPartitionsRDD[14] at map at <console>:38

In [32]:
val wordGroups = words.groupByKey.sortByKey(ascending = true)
wordGroups

wordGroups = ShuffledRDD[18] at sortByKey at <console>:40


ShuffledRDD[18] at sortByKey at <console>:40

In [33]:
wordGroups.count

11951

In [34]:
wordGroups.take(30).foreach(println)

(,CompactBuffer((tamingoftheshrew,1), (asyoulikeit,1), (merrywivesofwindsor,1), (comedyoferrors,1), (midsummersnightsdream,1), (twelfthnight,1), (loveslabourslost,1), (muchadoaboutnothing,1)))
(A,CompactBuffer((loveslabourslost,78), (midsummersnightsdream,39), (muchadoaboutnothing,31), (merrywivesofwindsor,38), (comedyoferrors,42), (asyoulikeit,34), (twelfthnight,47), (tamingoftheshrew,59)))
(ABOUT,CompactBuffer((muchadoaboutnothing,18)))
(ACT,CompactBuffer((asyoulikeit,22), (comedyoferrors,11), (tamingoftheshrew,12), (loveslabourslost,9), (muchadoaboutnothing,17), (twelfthnight,18), (merrywivesofwindsor,23), (midsummersnightsdream,9)))
(ADAM,CompactBuffer((asyoulikeit,16)))
(ADO,CompactBuffer((muchadoaboutnothing,18)))
(ADRIANA,CompactBuffer((comedyoferrors,85)))
(ADRIANO,CompactBuffer((loveslabourslost,111)))
(AEGEON,CompactBuffer((comedyoferrors,20)))
(AEMELIA,CompactBuffer((comedyoferrors,16)))
(AEMILIA,CompactBuffer((comedyoferrors,3)))
(AEacides,CompactBuffer((tamingoftheshrew,1)

В Scala можно использовать метод [Vector](http://www.scala-lang.org/api/current/index.html#scala.collection.immutable.Vector) для создания объекта (key, [(key,v), (key,v), (key,v)])

 `-fileNameCountTuple2._2` - только для лучшей сортировки (большие отрицательные значения внизу списка)

In [35]:
val iiFirstPass2 = wordGroups.mapValues { iterable => 
    val vect = iterable.toVector.sortBy { file_count_tup2 => 
        (-file_count_tup2._2, file_count_tup2._1)
    }
    vect.mkString(",")
}

iiFirstPass2 = MapPartitionsRDD[19] at mapValues at <console>:42


MapPartitionsRDD[19] at mapValues at <console>:42

In [36]:
iiFirstPass2.take(30).foreach(println)

(,(asyoulikeit,1),(comedyoferrors,1),(loveslabourslost,1),(merrywivesofwindsor,1),(midsummersnightsdream,1),(muchadoaboutnothing,1),(tamingoftheshrew,1),(twelfthnight,1))
(A,(loveslabourslost,78),(tamingoftheshrew,59),(twelfthnight,47),(comedyoferrors,42),(midsummersnightsdream,39),(merrywivesofwindsor,38),(asyoulikeit,34),(muchadoaboutnothing,31))
(ABOUT,(muchadoaboutnothing,18))
(ACT,(merrywivesofwindsor,23),(asyoulikeit,22),(twelfthnight,18),(muchadoaboutnothing,17),(tamingoftheshrew,12),(comedyoferrors,11),(loveslabourslost,9),(midsummersnightsdream,9))
(ADAM,(asyoulikeit,16))
(ADO,(muchadoaboutnothing,18))
(ADRIANA,(comedyoferrors,85))
(ADRIANO,(loveslabourslost,111))
(AEGEON,(comedyoferrors,20))
(AEMELIA,(comedyoferrors,16))
(AEMILIA,(comedyoferrors,3))
(AEacides,(tamingoftheshrew,1))
(AEgeon,(comedyoferrors,7))
(AEgle,(midsummersnightsdream,1))
(AEmilia,(comedyoferrors,4))
(AEsculapius,(merrywivesofwindsor,1))
(AGUECHEEK,(twelfthnight,2))
(ALL,(midsummersnightsdream,2),(tamingof

## Pattern Matching



* `filter(word => word.size > 0)` удаляем пустые строки (слова) `// #1`.
* `word.toLowerCase` конвертация всех значений в нижний регистр `// #2`.

In [38]:
val ii1 = sc.wholeTextFiles(shakespeare.toString).
    flatMap {
        case (location, contents) => 
            val words = contents.split("""\W+""").
                filter(word => word.size > 0)                      // #1
            val fileName = location.split(pathSeparator).last
            words.map(word => ((word.toLowerCase, fileName), 1))   // #2
    }.
    reduceByKey((count1, count2) => count1 + count2).
    map { 
        case ((word, fileName), count) => (word, (fileName, count)) 
    }.
    groupByKey.
    sortByKey(ascending = true).
    mapValues { iterable => 
        val vect = iterable.toVector.sortBy { 
            case (fileName, count) => (-count, fileName) 
        }
        vect.mkString(",")
    }

ii1 = MapPartitionsRDD[39] at mapValues at <console>:54


MapPartitionsRDD[39] at mapValues at <console>:54

# DataFrame

In [39]:
import org.apache.spark.sql.SQLContext

In [40]:
val sqlContext = new SQLContext(sc)

sqlContext = org.apache.spark.sql.SQLContext@27a9da47


org.apache.spark.sql.SQLContext@27a9da47

Конвертируем `RDD` в `DataFrame` функцией `sqlContext.createDataFrame` и методом `toDF`

In [41]:
val ii1DF = sqlContext.createDataFrame(ii1).toDF("word", "locations_counts")

ii1DF = [word: string, locations_counts: string]


[word: string, locations_counts: string]

`%%dataframe`

In [42]:
%%dataframe
ii1DF

word,locations_counts
a,"(loveslabourslost,507),(merrywivesofwindsor,494),(muchadoaboutnothing,492),(asyoulikeit,461),(tamingoftheshrew,445),(twelfthnight,416),(midsummersnightsdream,281),(comedyoferrors,254)"
abandon,"(asyoulikeit,4),(tamingoftheshrew,1),(twelfthnight,1)"
abate,"(loveslabourslost,1),(midsummersnightsdream,1),(tamingoftheshrew,1)"
abatement,"(twelfthnight,1)"
abbess,"(comedyoferrors,8)"
abbey,"(comedyoferrors,9)"
abbominable,"(loveslabourslost,1)"
abbreviated,"(loveslabourslost,1)"
abed,"(asyoulikeit,1),(twelfthnight,1)"
abetting,"(comedyoferrors,1)"


In [43]:
// супер правильный вариант
val ii = sc.wholeTextFiles(shakespeare.toString).
    flatMap {
        case (location, contents) => 
            val words = contents.split("""\W+""").
                filter(word => word.size > 0)                      // #1
            val fileName = location.split(pathSeparator).last
            words.map(word => ((word.toLowerCase, fileName), 1))   // #2
    }.
    reduceByKey((count1, count2) => count1 + count2).
    map { 
        case ((word, fileName), count) => (word, (fileName, count)) 
    }.
    groupByKey.
    sortByKey(ascending = true).
    map {                         
      case (word, iterable) =>    

        val vect = iterable.toVector.sortBy { 
            case (fileName, count) => (-count, fileName) 
        }

        // `Vector.unzip` - возвращает каждый элемент в отдельности 
        val (locations, counts) = vect.unzip  
        
        // считаем общее кол-во по всем элементам
        val totalCount = counts.reduceLeft((n1,n2) => n1+n2)
        
        (word, totalCount, locations, counts)
    }

ii = MapPartitionsRDD[54] at map at <console>:55


MapPartitionsRDD[54] at map at <console>:55

In [44]:
// зарегистрируем, как SQL таблицу
val iiDF = sqlContext.createDataFrame(ii).toDF("word", "total_count", "locations", "counts")
iiDF.cache
iiDF.registerTempTable("inverted_index")

iiDF = [word: string, total_count: int ... 2 more fields]


[word: string, total_count: int ... 2 more fields]

Получение схемы таблицы

In [45]:
iiDF.printSchema

root
 |-- word: string (nullable = true)
 |-- total_count: integer (nullable = false)
 |-- locations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- counts: array (nullable = true)
 |    |-- element: integer (containsNull = false)



In [46]:
%%SQL
SELECT word, total_count, locations[0] AS top_location, counts[0] AS top_count 
FROM inverted_index 

+-----------+-----------+--...


+-----------+-----------+----------------+---------+
|       word|total_count|    top_location|top_count|
+-----------+-----------+----------------+---------+
|          a|       3350|loveslabourslost|      507|
|    abandon|          6|     asyoulikeit|        4|
|      abate|          3|loveslabourslost|        1|
|  abatement|          1|    twelfthnight|        1|
|     abbess|          8|  comedyoferrors|        8|
|      abbey|          9|  comedyoferrors|        9|
|abbominable|          1|loveslabourslost|        1|
|abbreviated|          1|loveslabourslost|        1|
|       abed|          2|     asyoulikeit|        1|
|   abetting|          1|  comedyoferrors|        1|
+-----------+-----------+----------------+---------+
only showing top 10 rows



`%%SQL` не содержит возможностей конфигурации, но `%%DataFrame` имеет некоторые опции

In [47]:
%%dataframe

%%dataframe [arguments]
DATAFRAME_CODE

DATAFRAME_CODE can be any numbered lines of code, as long as the
last line is a reference to a variable which is a DataFrame.
    Option    Description                       
------    -----------                       
--limit   The number of records to return   
            (default: 10)                   
--output  The type of the output: html, csv,
            json (default: html)            


`WHERE`

In [48]:
val topLocations = sqlContext.sql("""
    SELECT word,  total_count, locations[0] AS top_location, counts[0] AS top_count
    FROM inverted_index 
    WHERE word LIKE '%love%' OR word LIKE '%hate%'
""")

topLocations = [word: string, total_count: int ... 2 more fields]


[word: string, total_count: int ... 2 more fields]

`%%dataframe` _magic_ с опциями

In [49]:
%%dataframe --limit 100
topLocations

word,total_count,top_location,top_count
beloved,11,tamingoftheshrew,4
cloven,1,loveslabourslost,1
cloves,1,loveslabourslost,1
glove,3,loveslabourslost,2
glover,1,merrywivesofwindsor,1
gloves,5,merrywivesofwindsor,3
hate,22,midsummersnightsdream,9
hated,6,midsummersnightsdream,4
hateful,5,midsummersnightsdream,3
hates,5,asyoulikeit,2


There's also a useful `show` method on `DataFrames`.

In [50]:
topLocations.show

+-------+-----------+--------------------+---------+
|   word|total_count|        top_location|top_count|
+-------+-----------+--------------------+---------+
|beloved|         11|    tamingoftheshrew|        4|
| cloven|          1|    loveslabourslost|        1|
| cloves|          1|    loveslabourslost|        1|
|  glove|          3|    loveslabourslost|        2|
| glover|          1| merrywivesofwindsor|        1|
| gloves|          5| merrywivesofwindsor|        3|
|   hate|         22|midsummersnightsd...|        9|
|  hated|          6|midsummersnightsd...|        4|
|hateful|          5|midsummersnightsd...|        3|
|  hates|          5|         asyoulikeit|        2|
| hateth|          1|midsummersnightsd...|        1|
|   love|        662|    loveslabourslost|      121|
|  loved|         38|         asyoulikeit|       13|
| lovely|         15|midsummersnightsd...|        7|
|  lover|         33|         asyoulikeit|       14|
| lovers|         31|midsummersnightsd...|    

In [51]:
topLocations.show(numRows = 40, truncate = false)

+--------+-----------+---------------------+---------+
|word    |total_count|top_location         |top_count|
+--------+-----------+---------------------+---------+
|beloved |11         |tamingoftheshrew     |4        |
|cloven  |1          |loveslabourslost     |1        |
|cloves  |1          |loveslabourslost     |1        |
|glove   |3          |loveslabourslost     |2        |
|glover  |1          |merrywivesofwindsor  |1        |
|gloves  |5          |merrywivesofwindsor  |3        |
|hate    |22         |midsummersnightsdream|9        |
|hated   |6          |midsummersnightsdream|4        |
|hateful |5          |midsummersnightsdream|3        |
|hates   |5          |asyoulikeit          |2        |
|hateth  |1          |midsummersnightsdream|1        |
|love    |662        |loveslabourslost     |121      |
|loved   |38         |asyoulikeit          |13       |
|lovely  |15         |midsummersnightsdream|7        |
|lover   |33         |asyoulikeit          |14       |
|lovers  |

**Самостоятельное задание:** 


* Найдите слова `glove`, `gloves`, `whate` и `whatever` и найдите документ, где они чаще всего встречаются и кол-во упоминаний
* Ваш запрос должен возвращать колонки word, total_count, top_location,	top_count

In [52]:
val sql1 = sqlContext.sql("""
    SELECT * FROM inverted_index
""")
sql1.show(10, false)

+-----------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+
|word       |total_count|locations                                                                                                                                       |counts                                  |
+-----------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+
|a          |3350       |[loveslabourslost, merrywivesofwindsor, muchadoaboutnothing, asyoulikeit, tamingoftheshrew, twelfthnight, midsummersnightsdream, comedyoferrors]|[507, 494, 492, 461, 445, 416, 281, 254]|
|abandon    |6          |[asyoulikeit, tamingoftheshrew, twelfthnight]                                                                                  

sql1 = [word: string, total_count: int ... 2 more fields]


[word: string, total_count: int ... 2 more fields]

## Больше о Pattern Matching 

```scala
{
    case (location, "") => 
        Array.empty[((String, String), Int)]  // Пустой массив
    case (location, contents) => 
        val words = contents.split("""\W+""")
        val fileName = location.split(pathSep).last
        words.map(word => ((word, fileName), 1))
}.
```


In [53]:
val stuff = Seq(1, 3.14159, 2L, 4.4F, ("one", 1), (404F, "boo"), ((11, 12), 21, 31), "hello")

stuff.foreach {
    case i: Int               => println(s"Found an Int:   $i")
    case l: Long              => println(s"Found a Long:   $l")
    case f: Float             => println(s"Found a Float:  $f")
    case d: Double            => println(s"Found a Double: $d")
    case (x1, x2) => 
        println(s"Found a two-element tuple with elements of arbitrary type: ($x1, $x2)")
    case ((x1a, x1b), _, x3) => 
        println(s"Found a three-element tuple with 1st and 3th elements: ($x1a, $x1b) and $x3")
    case default              => println(s"Found something else: $default")
}

Found an Int:   1
Found a Double: 3.14159
Found a Long:   2
Found a Float:  4.4
Found a two-element tuple with elements of arbitrary type: (one, 1)
Found a two-element tuple with elements of arbitrary type: (404.0, boo)
Found a three-element tuple with 1st and 3th elements: (11, 12) and 31
Found something else: hello


stuff = List(1, 3.14159, 2, 4.4, (one,1), (404.0,boo), ((11,12),21,31), hello)


List(1, 3.14159, 2, 4.4, (one,1), (404.0,boo), ((11,12),21,31), hello)

### unzip  

```scala
val (locations, counts) = vect.unzip  
```
[Vector.unzip](http://www.scala-lang.org/api/current/#scala.collection.immutable.Vector) 

In [54]:
val (a, (b, (c1, c2), d)) = ("A", ("B", ("C1", "C2"), "D"))
println(s" $a, $b, $c1, $c2, $d")

 A, B, C1, C2, D


a = A
b = B
c1 = C1
c2 = C2
d = D


D

## Scala's Object Model

### Classes vs. Instances

In [55]:
class IIRecord1(
    word: String, 
    total_count: Int, 
    locations: Array[String], 
    counts: Array[Int]) {
    
    /** CSV formatted string */
    override def toString: String = {
        val locStr = locations.mkString("[", ",", "]")  // i.e., "[a,b,c]"
        val cntStr = counts.mkString("[", ",", "]")  // i.e., "[1,2,3]"
        s"$word,$total_count,$locStr,$cntStr"
    }
}

new IIRecord1("hello", 3, Array("one", "two"), Array(1, 2))

defined class IIRecord1


hello,3,[one,two],[1,2]

### Objects

In [56]:
object MySparkJob {

    val greeting = "Hello Spark!"
    
    def main(arguments: Array[String]) = {
        println(greeting)
        
        // Create your SparkContext, etc., etc.
    }
}

defined object MySparkJob


### Case Classes

In [57]:
case class IIRecord(
    word: String, 
    total_count: Int = 0, 
    locations: Array[String] = Array.empty, 
    counts: Array[Int] = Array.empty) {

    /** 
     * toCSV.
     * Array.toString
     */
    override def toString: String = 
        s"""IIRecord($word, $total_count, $locStr, $cntStr)"""
    
    /** CSV-formatted string*/
    def toCSV: String = 
        s"$word,$total_count,$locStr,$cntStr"
        
    /** Return SON-formatted string */
    def toJSONString: String = 
        s"""{
        |  "word":        "$word", 
        |  "total_count": $total_count, 
        |  "locations":   ${toJSONArrayString(locations)},
        |  "counts"       ${toArrayString(counts, ", ")}
        |}
        |""".stripMargin

    private def locStr = toArrayString(locations)
    private def cntStr = toArrayString(counts)

    // "[_]" - означает, что не имеет значения, какой элемент;
    private def toArrayString(array: Array[_], delim: String = ","): String = 
        array.mkString("[", delim, "]")  // "[a,b,c]"

    private def toJSONArrayString(array: Array[String]): String =
        toArrayString(array.map(quote), ", ")
    
    private def quote(word: String): String = "\"" + word + "\""  
}

defined class IIRecord


**Применение**

In [58]:
val hello = new IIRecord("hello")
val world = new IIRecord("world!", 3, Array("one", "two"), Array(1, 2))

println("\n`toString` output:")
println(hello)
println(world)

println("\n`toJSONString` output:")
println(hello.toJSONString)
println(world.toJSONString)

println("\n`toCSV` output:")
println(hello.toCSV)
println(world.toCSV)


`toString` output:
IIRecord(hello, 0, [], [])
IIRecord(world!, 3, [one,two], [1,2])

`toJSONString` output:
{
  "word":        "hello", 
  "total_count": 0, 
  "locations":   [],
  "counts"       []
}

{
  "word":        "world!", 
  "total_count": 3, 
  "locations":   ["one", "two"],
  "counts"       [1, 2]
}


`toCSV` output:
hello,0,[],[]
world!,3,[one,two],[1,2]


hello = IIRecord(hello, 0, [], [])
world = IIRecord(world!, 3, [one,two], [1,2])


IIRecord(world!, 3, [one,two], [1,2])

In [59]:
val hello1 = new IIRecord("hello1")
val hello2 = IIRecord("hello2")

hello1 = IIRecord(hello1, 0, [], [])
hello2 = IIRecord(hello2, 0, [], [])


IIRecord(hello2, 0, [], [])

In [60]:
val hello2b = IIRecord.apply("hello2b")

hello2b = IIRecord(hello2b, 0, [], [])


IIRecord(hello2b, 0, [], [])

## Datasets and DataFrames


In [63]:
val sqlc = sqlContext
import sqlc.implicits._   // узнаем об это позже

sqlc = org.apache.spark.sql.SQLContext@27a9da47


org.apache.spark.sql.SQLContext@27a9da47

In [64]:
val iiDS = iiDF.as[IIRecord]
iiDS

iiDS = [word: string, total_count: int ... 2 more fields]


[word: string, total_count: int ... 2 more fields]

In [65]:
iiDS.show

+-----------+-----------+--------------------+--------------------+
|       word|total_count|           locations|              counts|
+-----------+-----------+--------------------+--------------------+
|          a|       3350|[loveslabourslost...|[507, 494, 492, 4...|
|    abandon|          6|[asyoulikeit, tam...|           [4, 1, 1]|
|      abate|          3|[loveslabourslost...|           [1, 1, 1]|
|  abatement|          1|      [twelfthnight]|                 [1]|
|     abbess|          8|    [comedyoferrors]|                 [8]|
|      abbey|          9|    [comedyoferrors]|                 [9]|
|abbominable|          1|  [loveslabourslost]|                 [1]|
|abbreviated|          1|  [loveslabourslost]|                 [1]|
|       abed|          2|[asyoulikeit, twe...|              [1, 1]|
|   abetting|          1|    [comedyoferrors]|                 [1]|
|abhominable|          1|  [loveslabourslost]|                 [1]|
|      abhor|          5|[asyoulikeit, com...|  

### Operator Syntax

Обратим внимание на синтаксис операторов в Scala

```scala
val m1_times_m2 = m1.*(m2)
val m1_times_m2 = m1 * m2
```

### Traits
_Traits_ == Java 8 _interfaces_ (но DS и Аналитикам это ни чего не говорит). Это объект, который является абстрактным типом, который используется для определения поведения, которое должны реализовывать классы

```scala
trait Logging {

    def log(level: Level, message: String): Unit = logger.log(level, message)
    
    private logger: Logger = ...
}

abstract class Service {
    def run(): Unit   // No body, so abstract!
}

class MyService extends Service with Logging {
    def run(): Unit = {
        log(INFO, "Staring MyService...")
        ...
        log(INFO, "Finished MyService")
    }
}
```


### Ranges
 [Range](http://www.scala-lang.org/api/current/index.html#scala.collection.immutable.Range), как и в Python range(0, 100), то в Scala => `1 until 100`, `2 to 200 by 3`. 


In [66]:
1 until 10

Range(1, 2, 3, 4, 5, 6, 7, 8, 9)

In [67]:
1 to 10

Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [68]:
1 to 10 by 3

Range(1, 4, 7, 10)

In [69]:
// придумаем себе данные на основе range
val rdd7 = sc.parallelize(1 to 50).
    map(i => (i, i%7)).
    groupBy{ case (i, seven) => seven }.
    sortByKey()
rdd7.take(7).foreach(println)

(0,CompactBuffer((7,0), (14,0), (21,0), (28,0), (35,0), (42,0), (49,0)))
(1,CompactBuffer((1,1), (8,1), (15,1), (22,1), (29,1), (36,1), (43,1), (50,1)))
(2,CompactBuffer((2,2), (9,2), (16,2), (23,2), (30,2), (37,2), (44,2)))
(3,CompactBuffer((3,3), (10,3), (17,3), (24,3), (31,3), (38,3), (45,3)))
(4,CompactBuffer((4,4), (11,4), (18,4), (25,4), (32,4), (39,4), (46,4)))
(5,CompactBuffer((5,5), (12,5), (19,5), (26,5), (33,5), (40,5), (47,5)))
(6,CompactBuffer((6,6), (13,6), (20,6), (27,6), (34,6), (41,6), (48,6)))


rdd7 = ShuffledRDD[101] at sortByKey at <console>:39


ShuffledRDD[101] at sortByKey at <console>:39

Пример использования

```scala
(1 to 100)
.map(i => i*i)
```

Создает пользовательность

```scala
res0: scala.collection.immutable.IndexedSeq[Int] = Vector(1, 4, 9, 16, 25, 36, 49, 64, 81, 100, ...)

```

### Scala иерархия типов

![Scala Type Hierarchy](http://docs.scala-lang.org/resources/images/classhierarchy.img_assist_custom.png)

In [70]:
// Option - тип с None и каким-то типом данных
val options = Seq(None, Some(2), Some(3), None, Some(5))

options.foreach { o =>
    println(o.getOrElse("None"))
}

None
2
3
None
5


options = List(None, Some(2), Some(3), None, Some(5))


List(None, Some(2), Some(3), None, Some(5))

In [71]:
options.foreach {
    case None    => println(None)
    case Some(i) => println(i)  
}

None
2
3
None
5


In [72]:
// фильтр для None
for {
    option <- options  
    value  <- option   
} println(value)

2
3
5


### Implicits

<a name="implicits"></a>

`reduceBykey`, `groupByKey` => `*ByKey`

In [74]:
// Простой класс без `toJSON`:
case class Person(name: String, age: Int = 0)

defined class Person


In [75]:
// создадим implicits объект
object implicits {

    // `implicit` - ключевое слово
    // На основе класса `Person`, мы создаем класс `PersonToJSONString`,
    // который содержит`toJSON`.
    implicit class PersonToJSONString(person: Person) {
        def toJSON: String = s"""{"name": ${person.name}, "age": ${person.age}}"""
    }
}

import implicits._        //импорт

val p = Person("Dean Wampler", 39)

// автоматически класс конвертируется в  `PersonToJSONString` и получает метод `toJSON`
p.toJSON

defined object implicits
p = Person(Dean Wampler,39)


{"name": Dean Wampler, "age": 39}

В Scala это позволяет пробрасывать методы из `RDD` в [DataFrame](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrame) API

In [76]:
val sqlc = sqlContext
import sqlc.implicits._  

sqlc = org.apache.spark.sql.SQLContext@27a9da47


org.apache.spark.sql.SQLContext@27a9da47

In [77]:
val wtc = iiDF.select($"word", $"total_count")
wtc.show

+-----------+-----------+
|       word|total_count|
+-----------+-----------+
|          a|       3350|
|    abandon|          6|
|      abate|          3|
|  abatement|          1|
|     abbess|          8|
|      abbey|          9|
|abbominable|          1|
|abbreviated|          1|
|       abed|          2|
|   abetting|          1|
|abhominable|          1|
|      abhor|          5|
|     abhors|          2|
|      abide|          5|
|     abides|          1|
|    ability|          2|
|     abject|          2|
|     abjure|          1|
|    abjured|          2|
|       able|          9|
+-----------+-----------+
only showing top 20 rows



wtc = [word: string, total_count: int]


[word: string, total_count: int]

Самое часто использование `implicits` при работе с колонками. `$"name"` - это метод в Scale`s"$foo"`, но при импорте `import sqlc.implicits._` му можем просто выбирать колонки, а не форматировать строку. 


In [78]:
import org.apache.spark.sql.functions._

Now we can use `min`, `max`, `avg`, etc.

In [79]:
// на примере min, max, avg
val mma = iiDF.select(min("total_count"), max("total_count"), avg("total_count"))
mma.show

+----------------+----------------+------------------+
|min(total_count)|max(total_count)|  avg(total_count)|
+----------------+----------------+------------------+
|               1|            5208|16.651743683350947|
+----------------+----------------+------------------+



mma = [min(total_count): int, max(total_count): int ... 1 more field]


[min(total_count): int, max(total_count): int ... 1 more field]